In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date,datetime

In [2]:
# Little function to keep track of code execution.
def Keep_Track():
    print("Executed successfully. " + datetime.now().strftime("%a %d %b @ %H:%M:%S") + "\n")

In [3]:
top_mkt = pd.read_excel('../TopMarket_Cap.xlsx')
tickers = list(top_mkt.symbol+'USDT')
path='../data'

In [4]:
def load_data(tickers,kline_interval,path):
    '''
    load feather file and return panel data with OHLCV
    '''
    l={}
    for tkrs in tickers:
        try:
            df = pd.read_feather(os.path.join(path,f'{tkrs}_{kline_interval}.feather'))
            df = df.set_index('Open Time')
            l[tkrs]=df
        except:
            print(f'{tkrs} is missing')
    df_panel = pd.concat(l).swaplevel().sort_index(level=[0,1])
    df_panel.index.set_names('Ticker',1,inplace=True)
    print(f'total ticker queried {len(tickers)}, total record returned {len(df_panel.index.get_level_values(1).unique())}')
    return df_panel

In [5]:
def cal_close_avg(df,w):
    #current close/avg
    d={}
    for i in w:
        ds = (df.Close/(df.Close.rolling(i).mean()))
        d[f'CurCloseDivAvg_{i}']=ds
    return pd.concat(d,axis=1)

In [6]:
def cal_volume_avg(df,w):
    #current volume/avg
    d={}
    for i in w:
        ds = (df.Volume/(df.Volume.rolling(i).mean()))
        d[f'CurVolumeDivAvg_{i}']=ds
    return pd.concat(d,axis=1)

In [7]:
def cal_bk_return(df,w):
    #period return up to now
    d={}
    for i in w:
        ds = df.Close.pct_change(i)
        d[f'BackReturn_{i}']=ds
    return pd.concat(d,axis=1)

In [8]:
def cal_fwd_return(df,w):
    #period forward return up to w
    d={}
    for i in w:
        ds = df.Close.pct_change(i).shift(-i)
        d[f'FwdReturn_{i}']=ds
    return pd.concat(d,axis=1)

In [9]:
def cal_high_low(df,w):
    #rolling high/rolling low
    d={}
    for i in w:
        ds=df.High.rolling(i).max()/df.Low.rolling(i).min()
        d[f'HighDivLow_{i}']=ds
    return pd.concat(d,axis=1)

In [10]:
def cal_max_mean(df,w):
    #rolling high/rolling mean
    d={}
    for i in w:
        ds=df.High.rolling(i).max()/df.Close.rolling(i).mean()
        d[f'MaxDivMean_{i}']=ds
    return pd.concat(d,axis=1)

In [11]:
def cal_min_mean(df,w):
    #rolling min/rolling mean
    d={}
    for i in w:
        ds=df.Low.rolling(i).min()/df.Close.rolling(i).mean()
        d[f'MinDivMean_{i}']=ds
    return pd.concat(d,axis=1)

In [12]:
def cal_max_mean_volume(df,w):
    #rolling high/rolling mean for volume
    d={}
    for i in w:
        ds=df.Volume.rolling(i).max()/df.Volume.rolling(i).mean()
        d[f'MaxDivMeanVolume_{i}']=ds
    return pd.concat(d,axis=1)

In [13]:
def cal_min_mean_volume(df,w):
    #rolling min/rolling mean for volume
    d={}
    for i in w:
        ds=df.Volume.rolling(i).min()/df.Volume.rolling(i).mean()
        d[f'MinDivMeanVolume_{i}']=ds
    return pd.concat(d,axis=1)

In [14]:
def cal_std(df,w):
    #rolling std 
    d={}
    for i in w:
        ds=df.Close.rolling(i).std()
        d[f'Std_{i}']=ds
    return pd.concat(d,axis=1)

In [15]:
def generate_factors(df_panel,factor_functions,w):
    '''
    generate factors on panel data with rolling window w
    '''
    l=[]
    for f in factor_functions:
        factor = df_panel.groupby('Ticker').apply(f,w)
        l.append(factor)
    return pd.concat(l,axis=1)

In [16]:
#load historical data and put them into panel data
df_panel = load_data(tickers=tickers,kline_interval='1HOUR',path=path)
columns_mapping={'Quote Asset Volume':'Turnover'}
df_panel =df_panel.rename(columns=columns_mapping)
df_panel.index = df_panel.index.rename(['DateTime','Ticker'])

DAIUSDT is missing
total ticker queried 29, total record returned 28


In [17]:
#register functions in the list for batch processing
factor_functions=[cal_close_avg,cal_bk_return,cal_high_low,cal_max_mean,cal_min_mean,cal_volume_avg,cal_max_mean_volume,cal_min_mean_volume,cal_std]
#calculate all factors
factors = generate_factors(df_panel,factor_functions,w=[2,5,10])
print(f'total categorical factors generated: {len(factor_functions)}')

total categorical factors generated: 9


In [18]:
factors

CurCloseDivAvg_2  CurCloseDivAvg_5  \
DateTime            Ticker                                         
2022-01-01 00:00:00 ADAUSDT                NaN               NaN   
                    ALGOUSDT               NaN               NaN   
                    ATOMUSDT               NaN               NaN   
                    AVAXUSDT               NaN               NaN   
                    BNBUSDT                NaN               NaN   
...                                        ...               ...   
2022-05-05 23:00:00 XMRUSDT           0.996623          0.998550   
                    XRPUSDT           1.006309          1.005345   
                    XTZUSDT           1.005659          1.005090   
                    ZECUSDT           1.005609          1.005126   
2022-05-06 00:00:00 APEUSDT           0.993446          0.990765   

                              CurCloseDivAvg_10  BackReturn_2  BackReturn_5  \
DateTime            Ticker                                                    
2022-01-01 00:00:00 ADAUSDT                 NaN           NaN           NaN   
                    ALGOUSDT                NaN           NaN           NaN   
                    ATOMUSDT                NaN           NaN           NaN   
                    AVAXUSDT                NaN           NaN           NaN   
                    BNBUSDT                 NaN           NaN           NaN   
...                                         ...           ...           ...   
2022-05-05 23:00:00 XMRUSDT            1.004962     -0.013842      0.023279   
                    XRPUSDT            0.999783      0.004197      0.012356   
                    XTZUSDT            0.998675      0.003630      0.008921   
                    ZECUSDT            1.010874      0.007223      0.028689   
2022-05-06 00:00:00 APEUSDT            0.990508     -0.003620      0.007305   

                              BackReturn_10  HighDivLow_2  HighDivLow_5  \
DateTime            Ticker                                                
2022-01-01 00:00:00 ADAUSDT             NaN           NaN           NaN   
                    ALGOUSDT            NaN           NaN           NaN   
                    ATOMUSDT            NaN           NaN           NaN   
                    AVAXUSDT            NaN           NaN           NaN   
                    BNBUSDT             NaN           NaN           NaN   
...                                     ...           ...           ...   
2022-05-05 23:00:00 XMRUSDT       -0.066004      1.015988      1.060423   
                    XRPUSDT       -0.047004      1.021898      1.037752   
                    XTZUSDT       -0.083947      1.016300      1.031405   
                    ZECUSDT       -0.053544      1.025101      1.064706   
2022-05-06 00:00:00 APEUSDT       -0.097192      1.038264      1.044448   

                              HighDivLow_10  MaxDivMean_2  ...  \
DateTime            Ticker                                 ...   
2022-01-01 00:00:00 ADAUSDT             NaN           NaN  ...   
                    ALGOUSDT            NaN           NaN  ...   
                    ATOMUSDT            NaN           NaN  ...   
                    AVAXUSDT            NaN           NaN  ...   
                    BNBUSDT             NaN           NaN  ...   
...                                     ...           ...  ...   
2022-05-05 23:00:00 XMRUSDT        1.132793      1.011577  ...   
                    XRPUSDT        1.086709      1.012871  ...   
                    XTZUSDT        1.131481      1.008084  ...   
                    ZECUSDT        1.126050      1.014423  ...   
2022-05-06 00:00:00 APEUSDT        1.166259      1.018941  ...   

                              CurVolumeDivAvg_10  MaxDivMeanVolume_2  \
DateTime            Ticker                                             
2022-01-01 00:00:00 ADAUSDT                  NaN                 NaN   
                    ALGOUSDT                 NaN                 N

In [ ]:
# generate target Y, the timeseries needs to be shift by rolling window w for prediction
fwd_return = df_panel.groupby('DateTime').apply(cal_fwd_return,w=[3,6,9])
fwd_return_rank = fwd_return.groupby('DateTime').apply(pd.DataFrame.rank)

In [ ]:
fwd_return_rank